<a href="https://colab.research.google.com/github/smbillah/ist526/blob/main/starter_code_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Essential Libraries

In [17]:
# python visualization libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
# from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
%matplotlib inline

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Steps to High Dimensional Data Reduction

## Loading data

### Loading large dataset from Google Drive
For large-scale datasets, it is not possible to load them from GitHub due to network traffics and latency. So, we need to load them from  Google Drive. Steps are shown below.

**Make sure you to comment the following blocks if you are loading data from GitHub. 


### Mounting Google Drive

In [18]:
import os

# mount drive
from google.colab import drive
drive.mount('/content/drive')


# edit this path if needed
my_path = '/content/drive/My Drive/Colab Notebooks/'

# change to this path
os.chdir(my_path)

# verify present working directory. It should be identical to 'my_path'
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


### Loading a .csv file from the 'data' folder in Google Drive

In [19]:
## assuming you have a 'data' folder in your 'My Drive/Colab Notebooks' folder, where a dataset (e.g., 'eurodist.csv') exists
## if you don't have the 'data' folder, create one

df = pd.read_csv('data/eurodist.csv')

display(df.head())
display(df.tail())

,city,Athens,Barcelona,Brussels,Calais,Cherbourg,Cologne,Copenhagen,Geneva,Gibraltar,...,Lisbon,Lyons,Madrid,Marseilles,Milan,Munich,Paris,Rome,Stockholm,Vienna
0,Athens,0,3313,2963,3175,3339,2762,3276,2610,4485,...,4532,2753,3949,2865,2282,2179,3000,817,3927,1991
1,Barcelona,3313,0,1318,1326,1294,1498,2218,803,1172,...,1305,645,636,521,1014,1365,1033,1460,2868,1802
2,Brussels,2963,1318,0,204,583,206,966,677,2256,...,2084,690,1558,1011,925,747,285,1511,1616,1175
3,Calais,3175,1326,204,0,460,409,1136,747,2224,...,2052,739,1550,1059,1077,977,280,1662,1786,1381
4,Cherbourg,3339,1294,583,460,0,785,1545,853,2047,...,1827,789,1347,1101,1209,1160,340,1794,2196,1588


,city,Athens,Barcelona,Brussels,Calais,Cherbourg,Cologne,Copenhagen,Geneva,Gibraltar,...,Lisbon,Lyons,Madrid,Marseilles,Milan,Munich,Paris,Rome,Stockholm,Vienna
16,Munich,2179,1365,747,977,1160,583,1104,591,2565,...,2507,724,2010,1109,331,0,821,946,1754,428
17,Paris,3000,1033,285,280,340,465,1176,513,1971,...,1799,471,1273,792,856,821,0,1476,1827,1249
18,Rome,817,1460,1511,1662,1794,1497,2050,995,2631,...,2700,1048,2097,1011,586,946,1476,0,2707,1209
19,Stockholm,3927,2868,1616,1786,2196,1403,650,2068,3886,...,3231,2108,3188,2428,2187,1754,1827,2707,0,2105
20,Vienna,1991,1802,1175,1381,1588,937,1455,1019,2974,...,2937,1157,2409,1363,898,428,1249,1209,2105,0


### Preprocessing 1: data cleaning

In [20]:
# removing non-numeric, unwanted columns, and rows with missing data

label =  df['city']
df = df.drop(['city'], axis = 1)


# removing rows with empty values
df.dropna(inplace=True)
display(df.head())

,Athens,Barcelona,Brussels,Calais,Cherbourg,Cologne,Copenhagen,Geneva,Gibraltar,Hamburg,...,Lisbon,Lyons,Madrid,Marseilles,Milan,Munich,Paris,Rome,Stockholm,Vienna
0,0,3313,2963,3175,3339,2762,3276,2610,4485,2977,...,4532,2753,3949,2865,2282,2179,3000,817,3927,1991
1,3313,0,1318,1326,1294,1498,2218,803,1172,2018,...,1305,645,636,521,1014,1365,1033,1460,2868,1802
2,2963,1318,0,204,583,206,966,677,2256,597,...,2084,690,1558,1011,925,747,285,1511,1616,1175
3,3175,1326,204,0,460,409,1136,747,2224,714,...,2052,739,1550,1059,1077,977,280,1662,1786,1381
4,3339,1294,583,460,0,785,1545,853,2047,1115,...,1827,789,1347,1101,1209,1160,340,1794,2196,1588


## Create X from the dataframe

In [21]:
## making X
X = df.values
print(X.shape, label.shape)

(21, 21) (21,)


# Q.4.2: Explain why MDS output is different than t-SNE. Also, does the MDS output look like the geographical map of different european cities (https://www.google.com/maps/@48.4702145,1.1137542,5z)? If so, why? 

In [26]:
from sklearn.manifold import MDS
from sklearn.manifold import TSNE

## use'dissimilarity'='precomputed'; otherwise 'eucledian' distance will be used by default
model1 = MDS(n_components=2, dissimilarity='precomputed')
model2 = TSNE(n_components=2,  metric='precomputed', perplexity = 30)

## run dimensionality reduction 
reduced_dims1 = model1.fit_transform(X1)
reduced_dims2 = model2.fit_transform(X2)

## create a subplot with nrow and ncol
nrow = 1
ncol = 2
figs = subplots.make_subplots(rows=nrow, cols=ncol)

## create individual plot first
fig1 = go.Scatter(x = reduced_dims1[:, 0], y = reduced_dims1[:, 1], mode="markers+text", name="mds_cityblock", text = label)
fig2 = go.Scatter(x = reduced_dims2[:, 0], y = reduced_dims2[:, 1], mode="markers+text", name="mds_euclidean", text = label)

## add individual plots to subplot
figs.add_trace(fig1, row=1, col=1) 
figs.add_trace(fig2, row=1, col=2) 

## adjust height and width of subplot
figs.update_layout(
  autosize=False,
  width=800 * ncol,
  height=800 * nrow,
)

# # visualize  subplot
figs.show()